In [1]:
import pandas as pd
import os,natsort,json

In [2]:
# need to clean up this block of code
csv_dir="logs"
program_file_dir="operations"
# get all program files that end with .py
program_files = [f for f in os.listdir(program_file_dir) if f.endswith(".py")]
program = program_files[0].split(".")[0]
same_program_csv_dir=os.path.join(csv_dir,program)
# samples = total number of runs for each program
samples = len(os.listdir(same_program_csv_dir))
op_to_func = {}

# TODO need to change the intersection to count
# loop through csv files in csv_dir
for program_ in program_files:
    program = program_.split(".")[0]
    same_program_csv_dir=os.path.join(csv_dir,program)
    common_funcs_per_op = set()
    c_func_to_count = {}
    for csv_file in os.listdir(same_program_csv_dir):
        df = pd.read_csv(os.path.join(same_program_csv_dir,csv_file))
        # merge two columns with "|" as separator
        df["full_func"] = df['Function'].astype(str) + "|" + df['Module'].astype(str)
        # loop through each function in the column 'full_func'
        for func in df["full_func"].values:
            # if function is not in the dictionary, add it
            if func not in c_func_to_count:
                c_func_to_count[func] = 1
            # else increment the count
            else:
                c_func_to_count[func] += 1

    for func in c_func_to_count:
        # i.e. present in more than half of the samples
        # if c_func_to_count[func] > samples//2:
        common_funcs_per_op.add(func)        

    op_to_func[program] = list(common_funcs_per_op)
    print(f"Common functions for {program} are {common_funcs_per_op}")

Common functions for ToTensor are {'futex_wake|libgomp.so.1', 'gomp_simple_barrier_wait|libgomp.so.1', 'func@0xce88e0|libtorch_cpu.so', 'gomp_team_barrier_wait_end|libgomp.so.1', '_Py_NewReference|python3.10', 'ImagingRawEncode|_imaging.cpython-310-x86_64-linux-gnu.so', '_PyObject_GetMethod|python3.10', '__memmove_avx_unaligned_erms|libc.so.6', 'tupledealloc|python3.10', 'lookdict_unicode_nodummy|python3.10', 'c10::function_ref<void (char**, long const*, long, long)>::callback_fn<at::TensorIteratorBase::loop_2d_from_1d<at::native::AVX2::copy_kernel(at::TensorIterator&, bool)::{lambda()#1}::operator()(void) const::{lambda()#7}::operator()(void) const::{lambda()#1}::operator()(void) const::{lambda()#1}::operator()(void) const::{lambda(char**long const*, long)#1}>(, signed char, at::native::AVX2::copy_kernel(at::TensorIterator&, bool)::{lambda()#1}::operator()(void) const::{lambda()#7}::operator()(void) const::{lambda()#1}::operator()(void) const::{lambda()#1}::operator()(void) const::{la

In [3]:
op_to_func

{'ToTensor': ['futex_wake|libgomp.so.1',
  'gomp_simple_barrier_wait|libgomp.so.1',
  'func@0xce88e0|libtorch_cpu.so',
  'gomp_team_barrier_wait_end|libgomp.so.1',
  '_Py_NewReference|python3.10',
  'ImagingRawEncode|_imaging.cpython-310-x86_64-linux-gnu.so',
  '_PyObject_GetMethod|python3.10',
  '__memmove_avx_unaligned_erms|libc.so.6',
  'tupledealloc|python3.10',
  'lookdict_unicode_nodummy|python3.10',
  'c10::function_ref<void (char**, long const*, long, long)>::callback_fn<at::TensorIteratorBase::loop_2d_from_1d<at::native::AVX2::copy_kernel(at::TensorIterator&, bool)::{lambda()#1}::operator()(void) const::{lambda()#7}::operator()(void) const::{lambda()#1}::operator()(void) const::{lambda()#1}::operator()(void) const::{lambda(char**long const*, long)#1}>(, signed char, at::native::AVX2::copy_kernel(at::TensorIterator&, bool)::{lambda()#1}::operator()(void) const::{lambda()#7}::operator()(void) const::{lambda()#1}::operator()(void) const::{lambda()#1}::operator()(void) const::{lam

In [4]:

def find_common_cpp_func(op_to_func):
    # find functions found in other ops
    common_funcs_to_op = {}
    # loop through all programs
    for python_func in op_to_func.keys():
        # loop through all functions in the program
        for func in op_to_func[python_func]:
            # loop through all other programs
            for other_python_func in op_to_func.keys():
                # skip if the program is the same as the current program
                if other_python_func == python_func:
                    continue
                # if the function is found in other programs
                if func in op_to_func[other_python_func]:
                    # add the function to the set of common functions
                    if func not in common_funcs_to_op:
                        common_funcs_to_op[func] = [python_func,other_python_func]
                    elif python_func not in common_funcs_to_op[func] or other_python_func not in common_funcs_to_op[func]:
                        common_funcs_to_op[func].append(other_python_func)
    return common_funcs_to_op

In [5]:
common_funcs_to_op = find_common_cpp_func(op_to_func)
print("C/C++ functions that can be found in multiple python operations:")
for func in common_funcs_to_op.keys():
    print(f"\t{func}\n\t\t->\t{common_funcs_to_op[func]}")

C/C++ functions that can be found in multiple python operations:
	futex_wake|libgomp.so.1
		->	['ToTensor', 'Normalize', 'ElasticTransform']
	gomp_simple_barrier_wait|libgomp.so.1
		->	['ToTensor', 'RandomErasing', 'Normalize', 'ElasticTransform', 'Lambda', 'LinearTransformation', 'GaussianBlur']
	gomp_team_barrier_wait_end|libgomp.so.1
		->	['ToTensor', 'RandomErasing', 'Normalize', 'ElasticTransform', 'Lambda', 'GaussianBlur']
	ImagingRawEncode|_imaging.cpython-310-x86_64-linux-gnu.so
		->	['ToTensor', 'PILToTensor', 'RandomGrayscale']
	_PyObject_GetMethod|python3.10
		->	['ToTensor', 'PILToTensor', 'GaussianBlur']
	__memmove_avx_unaligned_erms|libc.so.6
		->	['ToTensor', 'PILToTensor', 'ToPILImage', 'ColorJitter', 'Pad', 'RandomResizedCrop', 'RandomGrayscale', 'RandomVerticalFlip', 'GaussianBlur']
	lookdict_unicode_nodummy|python3.10
		->	['ToTensor', 'RandomGrayscale']
	c10::function_ref<void (char**, long const*, long, long)>::callback_fn<at::TensorIteratorBase::loop_2d_from_1d<at

In [6]:
json_string = {'op_to_func':op_to_func,'common_funcs_to_op':common_funcs_to_op}
with open('mapping_funcs.json', 'w') as outfile:
    json.dump(json_string, outfile,indent=4)